In [1]:
#导入SMAC包
import skopt
from skopt import forest_minimize
import warnings
warnings.filterwarnings("ignore")
import os 
import csv
for root, dirs, files in os.walk('dataset'):  
        print(root) #当前目录路径  
        print(files) #当前路径下所有非目录子文件
#进入dataset文件夹
os.chdir('dataset')

dataset
['ant-1.3.csv', 'ant-1.4.csv', 'ant-1.5.csv', 'ant-1.6.csv', 'ant-1.7.csv', 'camel-1.0.csv', 'camel-1.2.csv', 'camel-1.4.csv', 'camel-1.6.csv', 'ivy-1.1.csv', 'ivy-1.4.csv', 'ivy-2.0.csv', 'jedit-3.2.csv', 'jedit-4.0.csv', 'jedit-4.1.csv', 'jedit-4.2.csv', 'jedit-4.3.csv', 'log4j-1.0.csv', 'log4j-1.1.csv', 'log4j-1.2.csv', 'lucene-2.0.csv', 'lucene-2.2.csv', 'lucene-2.4.csv', 'poi-1.5.csv', 'poi-2.0.csv', 'poi-2.5.csv', 'poi-3.0.csv', 'synapse-1.0.csv', 'synapse-1.1.csv', 'synapse-1.2.csv', 'velocity-1.4.csv', 'velocity-1.5.csv', 'velocity-1.6.csv', 'xalan-2.4.csv', 'xalan-2.5.csv', 'xalan-2.6.csv', 'xalan-2.7.csv', 'xerces-1.2.csv', 'xerces-1.3.csv', 'xerces-1.4.csv']


In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import sklearn.model_selection as sk_model_selection
from sklearn import metrics


In [3]:
#写入文件函数
def writercsv(file_path,param,G):
    if os.path.isfile(file_path):
        with open(file_path,'a',newline='')as f:
            csv_write=csv.writer(f,dialect='excel')
            csv_write.writerow(G)
    else:
        with open(file_path,'w',newline='')as f:
            csv_write=csv.writer(f,dialect='excel')
            csv_write.writerow(param)
            csv_write.writerow(G)
#文件路径
file_path='..\SMAC_SVC_10.csv'
param=['Precision','Recall','F-measure','AUC','kernel','C','coef0']

In [4]:
#顺序读取文件进行实验
num=0
while num<39:
    #读取文件
    df1=pd.read_csv(files[num],header=0)
    df2=pd.read_csv(files[num+1],header=0)
    print(files[num],"->",files[num+1])
    #调参实验
    x_train=df1.iloc[:,0:19].values
    y_train=df1.iloc[:,20].values
    x_test=df2.iloc[:,0:19].values
    y_true=df2.iloc[:,20].values
    
    #特征选择
    from skfeature.function.statistical_based import CFS
    feature_index = CFS.cfs(x_train,y_train) # 在训练集上训练
    num_feature = 10 # 从原数据集中选择10个特征
    t_train = x_train[:,feature_index[0:num_feature]] # 转换训练集
    t_test = x_test[:,feature_index[0:num_feature]] # 转换测试集
    
    #定义model
    m=SVC(probability=True)
    #参数空间
    from skopt.space import Real,Integer,Categorical
    space=[#Categorical(('sigmoid','poly','rbf'),name='kernel'),
           Real(1.0,10.0,name='C'),
           Real(0.0,10.0,name='coef0')]
    #目标函数
    from skopt.utils import use_named_args
    @use_named_args(space)
    def SVM(**params): 
        m.set_params(**params)
        roc_auc=sk_model_selection.cross_val_score(m,t_train,y_train,scoring='roc_auc',cv=5)  
        return -roc_auc.mean()
    
    result=forest_minimize(SVM,space,n_calls=200,random_state=42,base_estimator='RF')
    #参数带回实验进行预测
    model=SVC(probability=True,kernel='sigmoid',C=result.x[0],coef0=result.x[1])
    model.fit(t_train,y_train)
    y_pred=model.predict(t_test)
    #评分
    G=[]
    probas=model.predict_proba(t_test)
    fpr,tpr,thresholds=roc_curve(y_true,probas[:,1])
    auc=metrics.auc(fpr,tpr)
    f_measure=metrics.f1_score(y_true,y_pred,average='weighted')
    recall=metrics.recall_score(y_true,y_pred,average='weighted')
    precision=metrics.precision_score(y_true,y_pred,average='weighted')
    G.append(precision)
    G.append(recall)
    G.append(f_measure)
    G.append(auc)
    G.append(result.x[0])
    G.append(result.x[1])
    #G.append(result.x[2])
    #G.append(result.x[2])
    #写入文件
    writercsv(file_path,param,G)
    print('写入文件成功')
    #版本变化
    num=num+1
    print(num)
    #项目变化
    if((num==4)or(num==8)or(num==11)or(num==16)or(num==19)or(num==22)or(num==26)or(num==29)or(num==32)or(num==36)):
        num=num+1
        print("项目变化",num)

ant-1.3.csv -> ant-1.4.csv
写入文件成功
1
ant-1.4.csv -> ant-1.5.csv
写入文件成功
2
ant-1.5.csv -> ant-1.6.csv
写入文件成功
3
ant-1.6.csv -> ant-1.7.csv
写入文件成功
4
项目变化 5
camel-1.0.csv -> camel-1.2.csv
写入文件成功
6
camel-1.2.csv -> camel-1.4.csv
写入文件成功
7
camel-1.4.csv -> camel-1.6.csv
写入文件成功
8
项目变化 9
ivy-1.1.csv -> ivy-1.4.csv
写入文件成功
10
ivy-1.4.csv -> ivy-2.0.csv
写入文件成功
11
项目变化 12
jedit-3.2.csv -> jedit-4.0.csv
写入文件成功
13
jedit-4.0.csv -> jedit-4.1.csv
写入文件成功
14
jedit-4.1.csv -> jedit-4.2.csv
写入文件成功
15
jedit-4.2.csv -> jedit-4.3.csv
写入文件成功
16
项目变化 17
log4j-1.0.csv -> log4j-1.1.csv
写入文件成功
18
log4j-1.1.csv -> log4j-1.2.csv
写入文件成功
19
项目变化 20
lucene-2.0.csv -> lucene-2.2.csv
写入文件成功
21
lucene-2.2.csv -> lucene-2.4.csv
写入文件成功
22
项目变化 23
poi-1.5.csv -> poi-2.0.csv
写入文件成功
24
poi-2.0.csv -> poi-2.5.csv
写入文件成功
25
poi-2.5.csv -> poi-3.0.csv
写入文件成功
26
项目变化 27
synapse-1.0.csv -> synapse-1.1.csv
写入文件成功
28
synapse-1.1.csv -> synapse-1.2.csv
写入文件成功
29
项目变化 30
velocity-1.4.csv -> velocity-1.5.csv
写入文件成功
31
velocity-1.5.csv -> 